## The demonstrative notebook for Hive assignments.

To run any HiveQL query in the notebook you should:
1. write the code of query into a separate file using `%%writefile [-a] <file>` magic,
2. execute this file in hive using `! hive -f <file>` command.

To make grading system check a task correctly, execution command must be in a separate cell.

### 1. Creation the database.

Firstly, create your Hive database. You can name the database whatever you want.

Let's drop database if it has already created.

In [1]:
%%writefile creation_db.hql

DROP DATABASE IF EXISTS demodb CASCADE;

Writing creation_db.hql


And now create it.

In [2]:
%%writefile -a creation_db.hql
CREATE DATABASE demodb LOCATION '/user/jovyan/somemetastore';

Appending to creation_db.hql


Finally, execute the file we filled earlier.

In [3]:
! hive -f creation_db.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.062 seconds
OK
Time taken: 0.524 seconds


On the real Hadoop-cluster where your submission will be checked we already have precreated Hive databases for all users. This helps to avoid database name conflicts. If you're the new user, the database will be created during your first submission of Hive assignment. The system won't allow you to create your own database on Hadoop-cluster so when you submit the final version of the task you shoud **remove or comment** all the lines related to database's dropping and creation. 

You can left all the lines with `USE` without any changes. The grading system will replace database's name to name of the precreated database. In assignments 2 and 3 you'll need to use `stackoverflow_` database. This database's name will not be changed by the grading system.

### 2. Creation the external table

Let us our source dataset have 2 collumns:
* ip-address,
* its subnet's mask.

For example:
```
148.45.113.216	255.255.255.248
203.98.141.0	255.255.255.240
183.168.36.0	255.255.255.128
111.157.172.232	255.255.255.248
80.46.87.0	255.255.255.0
247.248.233.0	255.255.255.128
```
Now we'll create the external table with 2 fields: ip and mask.

In [4]:
%%writefile exteral_table.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

USE demodb;
DROP TABLE IF EXISTS Subnets;

CREATE EXTERNAL TABLE Subnets (
    ip STRING,
    mask STRING
)
ROW FORMAT DELIMITED FIELDS TERMINATED BY  '\t'
STORED AS TEXTFILE
LOCATION '/data/subnets/ips';

Writing exteral_table.hql


In [5]:
! hive -f exteral_table.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
OK
Time taken: 1.071 seconds
OK
Time taken: 0.11 seconds
OK
Time taken: 0.93 seconds


### 3. Demo query on created table

Let's write a simpe query:
 > Compute avarage value of IPs for each subnet's mask.

In [6]:
%%writefile query.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
USE demodb;

Overwriting query.hql


In [7]:
%%writefile -a query.hql

SELECT AVG(counts.cnt)
FROM (
    SELECT mask, count(ip) as cnt
    FROM Subnets
    GROUP BY mask
) counts;

Appending to query.hql


In [8]:
! hive -f query.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
OK
Time taken: 1.052 seconds
Query ID = jovyan_20181113110505_cc5d3ac7-501f-4915-9d78-ffd05f07875d
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1542041219823_0001, Tracking URL = http://c5bd5fec03a1:8088/proxy/application_1542041219823_0001/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1542041219823_0001
Hadoop job information for

Please take into account that the grading system catch all output (both result and MapReduce logs) from the last cell of the notebook, so **don't** redirect any output from this cell to `/dev/null`.